In [6]:
from pixiedust.display.app import *

@PixieApp
class SelectBaseline():
    @route(connection="*", existing_baselines="true")
    def selectBaseline(self):
        return """
<div>
    Select a baseline to work with22222:
</div>
"""

In [9]:
from pixiedust.display.app import *
from pixiedust.utils import Logger
from pixiedust.utils.shellAccess import ShellAccess
from requests.auth import HTTPBasicAuth
import requests
import base64
import json
import pandas as pd
from pandas.io.json import json_normalize
from pixiedust.services.serviceManager import *
headers={
    "X-Watson-Origin" : "anything"
}
@PixieApp
@Logger()
class ConversationLogger(SelectBaseline):
    def fetchLogs(self):
        ws_id = "7ff0c3c2-a08a-4d3d-b13b-b6a9b62a4296"
        response = requests.get("https://gateway.watsonplatform.net/conversation/api/v1/workspaces/"+ ws_id + "/logs?version=2017-04-21", 
                                headers = headers,
                                auth=HTTPBasicAuth('50c95e08-34ef-453f-99f7-66d93df2641c','TlZczNCmdtlG')
                                ).json()['logs']
        
        self.debug("json log: {}".format(json.dumps(response[0], indent=2)))
        selectedCols = {'request.input.text':'input', 'response.intents':'intents', 'request_timestamp':'inputDate'}
        pdf = json_normalize(response)[selectedCols.keys()]
        pdf.columns= selectedCols.values()
        pdf=pdf[ (pd.notnull(pdf['input'])) & (pdf['input']!="")]
        pdf.drop_duplicates(subset='input',inplace=True)
        pdf['intent'] = pdf.apply (lambda row: row['intents'][0]['intent'], axis=1)
        pdf['confidence'] = pdf.apply (lambda row: row['intents'][0]['confidence'],axis=1)
        del pdf['intents']
        self.log = ShellAccess.sqlContext.createDataFrame(pdf)
        
    def getConnections(self):
        return getConnections("cloudant")
        
    @route(save='true')
    def saveToCloudant(self):
        self.debug("Saving to Cloudant")
        print("Saving")
        
    @route()
    def selectConnection(self):
        return """
<div>
  <div class="form-horizontal">
    <div class="form-group">
      <label for="connection{{prefix}}" class="control-label col-sm-2">Select a cloudant connection:</label>
      <div class="col-sm-5">
        <select class="form-control" id="connection{{prefix}}">
          {%for conn in this.getConnections() %}
              <option value="{{conn.name|escape}}">{{conn.name|escape}}</option>
          {%endfor%}
        </select>
      </div>
      <div class="col-sm-1">
        <button type="submit" class="btn btn-primary" pd_options="existing_baselines=true" pd_script="self.connection = $val(connection{{prefix}})">Work with Existing Baselines</button>
        <button type="submit" class="btn btn-primary" pd_options="new_baseline=true" pd_script="self.connection = $val(connection{{prefix}})">Create a new Baseline</button>
      </div>
    </div>
  </div>
</div>        
        """
    
    @route(connection="*", new_baseline="true")
    def newBaseline(self):
        return """
<div>
    <button pd_entity="log" pd_options="handlerId=dataframe;table_noschema=true" pd_target="json{{prefix}}" type="submit" class="btn btn-primary">Fetch Logs for last 90 days
        <pd_script>
            self.fetchLogs()
        </pd_script>
    </button>
    <button pd_target="json{{prefix}}" pd_entity="log" pd_options="handlerId=stashCloudant" type="submit" class="btn btn-primary">Save to Cloudant
    </button>
</div>
<div id="json{{prefix}}">
</div>        
        """
    
a = ConversationLogger()
a.run()

input,inputDate,intent,confidence
hello,2017-03-18T11:55:13.658Z,greetings,1.0
Midday,2017-03-18T12:57:44.136Z,goodbyes,0.249477755756
The nearest,2017-03-18T12:17:00.300Z,locate_amenity,0.999953621959
tu te fous de ma gueule!,2017-03-18T12:54:54.917Z,traffic_update,0.133984410432
Italian cuisine,2017-03-18T13:00:26.322Z,locate_amenity,0.997076963104
Turn on the headlights,2017-03-18T13:00:36.074Z,turn_on,0.999999905394
first,2017-03-18T11:59:15.450Z,not_specified,0.241838274938
What’s my ETA?,2017-03-18T13:01:16.416Z,traffic_update,0.999989350685
I don`t like mondays,2017-03-18T13:07:15.764Z,locate_amenity,0.512290643199
turn on,2017-03-18T13:07:20.589Z,turn_on,1.0


In [10]:
display(a.log)